## UTKFace experiments

In [ ]:
from utk_functions import get_distributed_utk_sets, get_lbfw_dataset, data_train_test_utk

# auxiliary data sets with specific property values:
all_dist = get_distributed_utk_sets()

# whole UTK data set, test data will be used for evaluation
X_train, X_test, y_train, y_test, _, _ = data_train_test_utk()

# attack data set
model_input = get_lbfw_dataset()

In [2]:
# example to train single utkface target/shadow model
from common.functions import get_lucasnet_model
import keras

target_dist = all_dist[4] 

#for dist in some_dist:
dist = target_dist
print(f"Training model for dist={dist.distribution}")
model = get_lucasnet_model(num_classes=2, input_shape=(64, 64, 3))
model.compile(
    optimizer=keras.optimizers.Adam(),#W(learning_rate=0.0001, weight_decay=0.0001),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)
hist = model.fit(
    dist.X_train, dist.y_train,
    batch_size=32,
    epochs=5,
    validation_data=(dist.X_test, dist.y_test)
)

Training model for dist=0.5


2025-03-17 23:40:39.503637: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/5
85/85 ━━━━━━━━━━━━━━━━━━━━ 9s 73ms/step - accuracy: 0.6801 - loss: 1.1648 - val_accuracy: 0.8279 - val_loss: 0.3830
Epoch 2/5
85/85 ━━━━━━━━━━━━━━━━━━━━ 6s 65ms/step - accuracy: 0.8428 - loss: 0.3451 - val_accuracy: 0.8435 - val_loss: 0.3490
Epoch 3/5
85/85 ━━━━━━━━━━━━━━━━━━━━ 5s 64ms/step - accuracy: 0.8752 - loss: 0.2986 - val_accuracy: 0.8570 - val_loss: 0.3194
Epoch 4/5
85/85 ━━━━━━━━━━━━━━━━━━━━ 5s 63ms/step - accuracy: 0.8945 - loss: 0.2540 - val_accuracy: 0.8327 - val_loss: 0.3746
Epoch 5/5
85/85 ━━━━━━━━━━━━━━━━━━━━ 5s 62ms/step - accuracy: 0.9121 - loss: 0.2156 - val_accuracy: 0.8813 - val_loss: 0.2786


## Black-box regression property inference

In [4]:
# shadow models must be trained before (see separate script)
import pandas as pd

# loading shadow model outputs as adversary's train/test data
adv_train = pd.concat([pd.read_csv(f"utkface/data/shadow_model_outputs/{dist}/train.csv") for dist in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]])
adv_test = pd.concat([pd.read_csv(f"utkface/data/shadow_model_outputs/{dist}/test.csv") for dist in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]])

adv_y = adv_train["y"]
adv_X = adv_train.drop(columns=["y"])
adv_y_test = adv_test["y"]
adv_X_test = adv_test.drop(columns=["y"])

In [5]:
from utk_functions import utk_adversary

adv = utk_adversary()
adv.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.MeanSquaredError(), metrics=[keras.metrics.R2Score()])

In [ ]:
# train adversary
checkpoint_filepath = 'utkface/models/manual_tuning_checkpoints-2/keras.weights.h5'
history = adv.fit(
    adv_X,
    adv_y,
    epochs=200,
    validation_data=(adv_X_test, adv_y_test),
    callbacks=[
        keras.callbacks.EarlyStopping('val_r2_score', mode='max', patience=25, verbose=1),
        keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_filepath,
            save_weights_only=True,
            monitor='val_r2_score',
            mode='max',
            save_best_only=True)
    ])

In [ ]:
adv.load_weights('utkface/models/manual_tuning_checkpoints/keras.weights.h5')
adv.evaluate(adv_X_test, adv_y_test)

In [ ]:
adv.save('utkface/models/bb-adversary.keras')

In [6]:
# loading adversary
adv = utk_adversary()
adv.load_weights('utkface/models/adv_v2_0.63_r2.keras')

/export/home/stock/anaconda3/envs/pia-tf/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [15]:
# adversary attacks target model
# black-box: adversary gets output of target model
output = model.predict(model_input)
formatted_input = output[:, 0].reshape(1, output.shape[0])
adv_out = adv(formatted_input).numpy().flatten()[0]
# this output is predicted property value of the target model
print(f"true property value: {target_dist.distribution}, predicted: {round(float(adv_out),2)}")

414/414 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step
true property value: 0.5, predicted: 0.61


## White-box regression property inference

In [16]:
# white-box: advesary gets model weights as inputs
# shadow models must be trained (and saved) first 
utk_base_path = "utkface/models/shadow_models"

In [18]:
# load example target model for adversary:
from common.whitebox_utils import do_read_single_model_params

m_utk = keras.models.load_model(f'{utk_base_path}/0.1/train/0.keras')
adv_inputs = do_read_single_model_params(m_utk)

In [19]:
# load model weights as train/test data set for adversary 
from common.whitebox_utils import get_dataset

ds_utk = get_dataset(200, utk_base_path, 'train')
ds_test_utk = get_dataset(50, utk_base_path, 'test')

In [20]:
ds_utk_batched = ds_utk.shuffle(1800).batch(32, drop_remainder=True)
ds_test_utk_batched = ds_test_utk.batch(8)

In [25]:
from keras import layers
from common.whitebox_utils import *

def get_wb_meta_clf(adv_inputs):
    inputs = []
    kernel_splitters = []
    bias_splitters = []

    phi_layers = []
    for i in range(len(adv_inputs)):
        for my_input in adv_inputs[i]:
            keras_input = keras.Input(shape=my_input.shape, name=transform_input_name(my_input.path, i))
            inputs.append(keras_input)
            if my_input.path.find("conv") >= 0 and my_input.path.find("kernel") >= 0:
                kernel_splitters.append(ConvSplitter(my_input.shape[3])(keras_input))
            elif my_input.path.find("bias") >= 0:
                bias_splitters.append(BiasSplitter(my_input.shape[0])(keras_input))
            else:
                kernel_splitters.append(WeightsSplitter(my_input.shape[1])(keras_input))

    phi_layers = []
    for kernels, biases in zip(kernel_splitters, bias_splitters):
        channel_outputs = []
        for w, b in zip(kernels, biases):
            flatten_w = layers.Flatten()(w)
            reshaped_b = layers.Reshape((1,))(b)
            concat = layers.concatenate([flatten_w, reshaped_b])
            phi = layers.Dense(20, activation="relu")(concat)
            channel_outputs.append(phi)
        my_phi = layers.Add()(channel_outputs)
        phi_layers.append(my_phi)

    x = layers.BatchNormalization()(layers.concatenate(phi_layers))
    x = layers.Dropout(0.15)(x)
    x = layers.Dense(30, activation='relu')(x)

    #x = layers.Dropout(0.2)(x)
    output = layers.Dense(1)(x)

    return keras.Model(inputs=inputs, outputs=output)

In [26]:
# train white-box adversary
from common.whitebox_utils import get_wb_meta_clf

meta_clf = get_wb_meta_clf(adv_inputs)
meta_clf.compile(optimizer=keras.optimizers.SGD(), loss=keras.losses.MeanAbsoluteError(), metrics=[keras.metrics.R2Score()])

In [ ]:
checkpoint_filepath = "utkface/models/manual_checkpoints/wb-meta/best.weights.h5"

currentmax = 0.4
for i in range(50): # multiple attempts
    meta_clf = get_wb_meta_clf(adv_inputs)
    meta_clf.compile(optimizer=keras.optimizers.SGD(), loss=keras.losses.MeanAbsoluteError(), metrics=[keras.metrics.R2Score()])
    history = meta_clf.fit(
        ds_utk_batched,
        epochs=70,
        validation_data=ds_test_utk_batched,
        callbacks=[
            keras.callbacks.EarlyStopping('val_r2_score', patience=5),
            keras.callbacks.ModelCheckpoint(
                filepath=checkpoint_filepath,
                save_weights_only=True,
                monitor='val_r2_score',
                mode='max',
                save_best_only=True)
        ])
    newmax = max(history.history['val_r2_score'])
    if newmax > currentmax:
        print(f"new max r2: {newmax}")
        currentmax = newmax
        meta_clf.load_weights("utkface/models/manual_checkpoints/wb-meta/best.weights.h5")
        meta_clf.save_weights(f"utkface/models/wb_adv_utk_{round(newmax, 2)}_test_r2.keras")

In [ ]:
meta_clf.load_weights(f"utkface/models/wb_adv_utk_{round(newmax, 2)}_test_r2.weights.h5")

In [ ]:
meta_clf.evaluate(ds_test_utk_batched)